In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/Lib_files')

Mounted at /content/gdrive


In [ ]:
''' DEBUG'''
#!pip install -Uqq ipdb
#import ipdb
#%pdb on

' DEBUG'

In [ ]:
# GPU bit!
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)#'''

Mon Jul 11 15:55:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#GPU 2
import torch
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
cuda0 = torch.device('cuda:0')#'''

1
Tesla V100-SXM2-16GB


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
"### CIFAR10 FRAMEWORK K-FAC solver"
import torch
import torchvision
import torchvision.transforms as transforms
import os
import time
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from data_utils import get_dataloader
from network_utils_for_vgg import get_network

from RS_KFAC_optimizer_file import RS_KFACOptimizer

In [ ]:

from torch.utils.data.dataloader import default_collate

torch.multiprocessing.set_start_method('spawn')

torch.backends.cudnn.enabled = True 
torch.backends.cudnn.benchmark = True

# ------------------------------
# ---- Training parameters -----
n_epochs = 50
opti_type = 'RS_KFAC'
# l_rate = 0.01;
def l_rate_function(epoch_n, iter_n):
    if epoch_n == 1:
        if iter_n < 3:
            return 0.3
        else:
            return 0.3
    elif epoch_n == 2:
        return 0.1
    elif epoch_n >= 3 and epoch_n < 13:
        return 0.1
    elif epoch_n >= 13 and epoch_n < 18:
        return 0.03
    elif epoch_n >= 18 and epoch_n < 27:
        return 0.01
    elif epoch_n >= 27 and epoch_n < 40:
        return 0.003
    elif epoch_n >= 40:
        return 0.001

for random_seed in [31, 32, 33, 34, 35, 36, 37, 38, 39, 40]:
  random_seed = random_seed + 1000
  print('Doing random seed {}... \n'.format(random_seed))

  kfac_clip = 7e-2; #KFAC_damping = 1e-01 #3e-02; 
  stat_decay = 0.95 #0.95

  momentum = 0.0
  WD = 7e-4 
  lambdaa = 0.0 
  batch_size_train = batch_size_test = 256
  # ====================================================


  ###################################### SCHEDULES ###############################
  # dict to have schedule! eys are epochs: key map to frequency, stuff only changes at keys and then stays constant.
  KFAC_matrix_update_frequency_dict = {0: 10, 5: 10, 10: 10, 20: 10, 22: 10, 50: 10}
  KFAC_matrix_invert_frequency_dict = {0: 50, 5: 50, 10: 50, 20: 30, 22: 30, 50: 30}

  KFAC_damping_dict = {0: 1e-01, 7:1e-01, 25: 5e-02, 35: 1e-02}
  
  rsvd_rank_dict = {0: 220, 7:220, 15:230, 20: 230, 22: 230, 30:230, 40:230}
  oversampling_parameter_dict = {0: 10, 7:10, 10:10, 20: 10, 22: 11, 30:12, 40:12}
  rsvd_niter_dict = {0: 4, 7:4, 10:4} #, 20: 10, 25: 18, 30:10, 40:10}

  ################################ END SCHEDULES #################################

  KFAC_matrix_update_frequency = KFAC_matrix_update_frequency_dict[0]
  KFAC_matrix_invert_frequency = KFAC_matrix_invert_frequency_dict[0]
  KFAC_damping = KFAC_damping_dict[0]
  
  rsvd_rank = rsvd_rank_dict[0]
  oversampling_parameter = oversampling_parameter_dict[0]
  rsvd_niter = rsvd_niter_dict[0]

  # ONLY FOR SAVED FILE NAME: beta1 and beta2 are just 2 channels for filename ==
  beta1 = WD
  beta2 = KFAC_damping

  log_interval = 500000000 
  basic_path = '/content/gdrive/My Drive/Randomized_KFAC_project/results_{}'.format(opti_type)
  error_write_path = '/content/gdrive/My Drive/Randomized_KFAC_project/err_{}_CIFAR10'.format(opti_type)

  #rsvd paramaters
  torch.manual_seed(random_seed)

  #------------------------------------------------------------------------------
  #--------------------------- DATA LOADERS -------------------------------------
  def collation_fct(x):
    return  tuple(x_.to(cuda0) for x_ in default_collate(x))
  #------------------------------------------------------------------------------

  train_loader, test_loader = get_dataloader(dataset = 'cifar10', train_batch_size = batch_size_train,
                                          test_batch_size = batch_size_test, num_workers = 0, 
                                          collation_fct = collation_fct, root='./data_CIFAR10')

  classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  #------------------------------------------------------------------------------
  #-------------------------------- building the NET ----------------------------

  ###### VGG16 using the construction on alecwangcq github in KFAC project ###############
  network = get_network('vgg16_bn',
                    #depth = 19,
                    num_classes = 10,
                    #growthRate = 12,
                    #compressionRate = 2,
                    widen_factor = 1)#,
                    #dropRate = 0.05)

  ######################### s.t. we can add dropout #########################
  network.classifier = nn.Sequential(
      nn.Linear(512, 512),
      nn.ReLU(True),
      nn.Dropout(), # p =0.5  by default
      nn.Linear(512, 10),
  )

  #################### end declaring net VGG16bn ###########################################

  network.to(cuda0)
  softmax_crossentropy = nn.CrossEntropyLoss() # since the net returns non-softmaxed output we need to use this loss which is softmax crossentropy
  softmax_crossentropy_test = nn.CrossEntropyLoss(size_average = False)
  
  print('the number of parameters is {}'.format(sum([p.numel() for p in network.parameters()])))
  # -------------------------------------------

  # --------------------------------- DEFINE THE OPTIMISER ----------------------
  # -----------------------------------------------------------------------------
  def regularized_loss_fct(output,target,network, lambdaa):
      KFAC_matrix_loss = softmax_crossentropy(output,target)
      l2_reg = torch.tensor(0., device = cuda0)
      for param in network.parameters():
          l2_reg += torch.norm(param)
      KFAC_matrix_loss += lambdaa * l2_reg
      
      loss_for_gradient = softmax_crossentropy(output,target)
      l2_reg = torch.tensor(0., device = cuda0)
      for param in network.parameters():
          l2_reg += torch.norm(param)
      loss_for_gradient += lambdaa * l2_reg
      
      return KFAC_matrix_loss, loss_for_gradient, l2_reg


  optimizer = RS_KFACOptimizer(network, lr_function = l_rate_function, momentum = momentum, stat_decay = stat_decay, 
                            kl_clip = kfac_clip, damping = KFAC_damping, 
                            weight_decay = WD, TCov = KFAC_matrix_update_frequency,
                            TInv = KFAC_matrix_invert_frequency, 
                            rsvd_rank = rsvd_rank, oversampling_parameter = oversampling_parameter, rsvd_niter = rsvd_niter)

  scheduler = None
  #scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda = [lambda1, lambda2])

  # ------------------------------------------------------------------------------

  # track performance of parameters and progress
  train_losses = []; train_losses_per_epoch = []; train_accuracy_per_epoch = []

  train_accuracy = []; test_accuracy = []; time_per_epoch_ = []; time_per_iter = []
  train_counter = []; test_losses = []
  test_counter = [i * len(train_loader.dataset) for i in range(n_epochs + 1)]

  def save_data_():
        # basic_path = os.path.join(basic_path, '/')
        train_losses_cpu = []; train_losses_per_epoch_cpu = []; train_accuracy_per_epoch_cpu = []
        train_accuracy_cpu = []; test_accuracy_cpu = []; time_per_epoch__cpu = []; time_per_iter_cpu = []
        test_losses_cpu = []

        '''PER-EPOCH GUYS CONVERSION TO CPU'''
        for trlpe,traccpe,timpe,teacc,tel in zip(train_losses_per_epoch, train_accuracy_per_epoch, time_per_epoch_, 
                                                              test_accuracy, test_losses):
          train_losses_per_epoch_cpu.append(trlpe.cpu()); train_accuracy_per_epoch_cpu.append(traccpe.cpu()); 
          time_per_epoch__cpu.append(timpe)
          test_accuracy_cpu.append(teacc.cpu()); test_losses_cpu.append(tel.cpu()); # these are per epoch by definition
        
        '''PER-ITERATION GUYS CONVERSION TO CPU'''
        for trl,tim in zip(train_losses, time_per_iter):
          train_losses_cpu.append(trl.cpu()); time_per_iter_cpu.append(tim);
        for tracc in train_accuracy: # apperently train accuracy per iter is wrongly saved only per epoch.... because it's saved only when logged
          train_accuracy_cpu.append(tracc.cpu());
        
        print('Saving data...\n Lengths of SAVED lists are {}, {}, {}, {}, {}, {}, {}, {}'.format(len(train_losses_cpu), len(train_losses_per_epoch_cpu), len(train_accuracy_per_epoch_cpu),
                                                                          len(train_accuracy_cpu), len(test_accuracy_cpu), len(time_per_epoch__cpu),
                                                                          len(time_per_iter_cpu), len(test_losses_cpu) ))
        print('Lengths of input lists are {}, {}, {}, {}, {}, {}, {}, {}'.format(len(train_losses), len(train_losses_per_epoch), len(train_accuracy_per_epoch),
                                                                          len(train_accuracy), len(test_accuracy), len(time_per_epoch_),
                                                                          len(time_per_iter), len(test_losses) ))
        np.save(os.path.join(basic_path,
                            '{}_{}_{}_{}_{}_{}_run{}_{}'.format(opti_type, batch_size_train,
                                                        beta1,
                                                        beta2, l_rate_function(40,40), stat_decay, 
                                                        random_seed, 'train_losses')),
                train_losses_cpu)
        np.save(os.path.join(basic_path,
                            '{}_{}_{}_{}_{}_{}_run{}_{}'.format(opti_type, batch_size_train,
                                                        beta1,
                                                        beta2, l_rate_function(40,40), stat_decay,
                                                        random_seed, 'time_per_iter')),
                time_per_iter_cpu)

        np.save(os.path.join(basic_path,
                            '{}_{}_{}_{}_{}_{}_run{}_{}'.format(opti_type, batch_size_train,
                                                        beta1,
                                                        beta2, l_rate_function(40,40), stat_decay ,
                                                        random_seed, 'train_losses_per_epoch')),
                train_losses_per_epoch_cpu)
        np.save(os.path.join(basic_path,
                            '{}_{}_{}_{}_{}_{}_run{}_{}'.format(opti_type, batch_size_train,
                                                        beta1,
                                                        beta2, l_rate_function(40,40), stat_decay, 
                                                        random_seed, 'train_accuracy')),
                train_accuracy_cpu)
        np.save(os.path.join(basic_path,
                            '{}_{}_{}_{}_{}_{}_run{}_{}'.format(opti_type, batch_size_train,
                                                        beta1,
                                                        beta2, l_rate_function(40,40), stat_decay,
                                                        random_seed, 'train_accuracy_per_epoch')),
                train_accuracy_per_epoch_cpu)
        np.save(os.path.join(basic_path,
                            '{}_{}_{}_{}_{}_{}_run{}_{}'.format(opti_type, batch_size_train,
                                                        beta1,
                                                        beta2, l_rate_function(40,40), stat_decay, 
                                                        random_seed, 'time_per_epoch_')),
                time_per_epoch__cpu)
        np.save(os.path.join(basic_path,
                            '{}_{}_{}_{}_{}_{}_run{}_{}'.format(opti_type, batch_size_train,
                                                        beta1,
                                                        beta2, l_rate_function(40,40), stat_decay,
                                                        random_seed, 'test_accuracy')),
                test_accuracy_cpu)
        np.save(os.path.join(basic_path,
                            '{}_{}_{}_{}_{}_{}_run{}_{}'.format(opti_type, batch_size_train,
                                                        beta1,
                                                        beta2, l_rate_function(40,40), stat_decay, 
                                                        random_seed, 'test_losses')),
                test_losses_cpu)
          
  def train(epoch, step_counter, log_interval = log_interval):
      network.train()
      correct = 0
      time_epoch = 0
      optimizer.epoch_number = epoch
      # previous_step = np.array([0])
      for batch_idx, (data, target) in enumerate(train_loader):
          step_counter = step_counter + 1
          start = time.time()
          optimizer.zero_grad()
          #data = data.double()
          output = network(data)
          pred = output.data.max(1, keepdim=True)[1]
          correct += pred.eq(target.data.view_as(pred)).sum()

          KFAC_matrix_loss, loss_for_gradient, l2_reg = regularized_loss_fct(output, target, network, lambdaa)
          #t1 = time.time()
          
          # update network weights
          optimizer.zero_grad()
          
          ''' assemble <KFAC matrix> loss to compute KFAC matrix'''
          if optimizer.steps % KFAC_matrix_update_frequency == 0:
              optimizer.acc_stats = True
              KFAC_matrix_loss.backward(retain_graph=True)
          optimizer.acc_stats = False
      
          ''' compute gradient of <Policy loss> (precond by KFAC^{-1}) and then take step'''
          ''' also need to compute and return the gradient for TRUEish F^{-1}g computation'''
          optimizer.zero_grad()
          loss_for_gradient.backward()
          KFAC_direction = optimizer.step(epoch, error_write_path)
          loss_value = loss_for_gradient.detach()
          end = time.time()

          time_per_iter.append(start - end)
          time_epoch = time_epoch + (end - start)
          train_losses.append(loss_value)
          if scheduler == None:
              pass
          else:
              scheduler.step()

          if batch_idx % log_interval == 0:
              train_counter.append(
                  (batch_idx * 64) + ((epoch - 1) * len(train_loader.dataset)))
              # change the saving path
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(train_loader.dataset),
                        100. * batch_idx / len(train_loader), loss_value))
              print('param norm: {}'.format(l2_reg))

      accc = 100. * correct / len(train_loader.dataset)
      train_accuracy.append(accc)
      time_per_epoch_.append(time_epoch)
      train_losses_per_epoch.append(loss_value)
      train_accuracy_per_epoch.append(accc)
      return step_counter

  def test():
      network.eval()
      test_loss = 0
      correct = 0
      with torch.no_grad():
          #print_idx = 0
          for data, target in test_loader:
              #print_idx = print_idx + 1; print('We are at {} index'.format(print_idx))
              #data = data.double()
              output = network(data)
              test_loss += softmax_crossentropy_test(output, target).detach()
              pred = output.data.max(1, keepdim=True)[1]
              #ipdb.set_trace(context = 7)
              correct += pred.eq(target.data.view_as(pred)).sum()
      test_loss /= len(test_loader.dataset)
      test_losses.append(test_loss)
      accc = 100. * correct / len(test_loader.dataset)
      test_accuracy.append(accc)
      print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(test_loader.dataset),
          100. * correct / len(test_loader.dataset)))
  t1_test = time.time()
  print('Started test before training')
  test()
  t2_test = time.time()
  print('Finished test before training in {}s'.format(t2_test - t1_test))
  print('\n m_aa just after testing (should be empty if we did nto accumulate stats during traning - this si correct)', optimizer.m_aa, '\n')
  step_counter = 0
  total_time_in_s = 0
  total_time_in_s_list = []
  for epoch in range(1, n_epochs + 1):
      # SET HYPER-OPTIMIZER PARAMETERS ACCODRING TO SHCEDULE
      if epoch in KFAC_matrix_update_frequency_dict:
        optimizer.TCov =  KFAC_matrix_update_frequency_dict[epoch]
      if epoch in KFAC_matrix_invert_frequency_dict:
        optimizer.TInv = KFAC_matrix_invert_frequency_dict[epoch]
      if epoch in KFAC_damping_dict: 
        optimizer.param_groups[0]['damping'] = KFAC_damping_dict[epoch]
      #if epoch in KFAC_damping_type_dict: 
      #  optimizer.damping_type = KFAC_damping_type_dict[epoch]
      if epoch in rsvd_rank_dict:
        optimizer.rsvd_rank = rsvd_rank_dict[epoch]
      if epoch in oversampling_parameter_dict:
        optimizer.oversampling_parameter = oversampling_parameter_dict[epoch]
      if epoch in rsvd_niter_dict:
        optimizer.rsvd_niter = rsvd_niter_dict[epoch]

      # END IMPLEMETING SCHEDULE for OPTIMIZER HYPERPARAMTERS

      t1 = time.time()
      step_counter = train(epoch, step_counter)
      t2 = time.time()
    
      test()
      total_time_in_s = total_time_in_s + (t2-t1)
      total_time_in_s_list.append(total_time_in_s)
      ############ SAVE DATA #####################
      save_data_()
      np.save(os.path.join(basic_path, '{}_{}_{}_{}_{}_{}_run{}_{}'.format(opti_type, batch_size_train,
                                                        beta1, beta2, l_rate_function(40,40), stat_decay, 
                                                        random_seed, 'total_train_time')),
                    total_time_in_s_list)
      ######### END SAVE DATA ####################
      print('Total time: {}s'.format(total_time_in_s))
  save_data = True
print('Done!')

Doing random seed 1031... 



  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data_CIFAR10/cifar-10-python.tar.gz to ./data_CIFAR10
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


the number of parameters is 14990922
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1033: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/content/gdrive/My Drive/Colab Notebooks/Lib_files/kfac_rsvd_for_vgg16_bn_improved_inversion_FC_project_adaptive_damping_no_damping_type_schedule.py:249: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  p.data.add_(-group['lr'], d_p)


Streaming output truncated to the last 5000 lines.
Saving data...
 Lengths of SAVED lists are 5488, 28, 28, 28, 28, 28, 5488, 28
Lengths of input lists are 5488, 28, 28, 28, 29, 28, 5488, 29
Total time: 910.4526376724243s
Train Epoch: 29 [0/50000 (0%)]	Loss: 0.021056
param norm: 1894.3963623046875

Test set: Avg. loss: 0.3561, Accuracy: 9243/10000 (92%)

Saving data...
 Lengths of SAVED lists are 5684, 29, 29, 29, 29, 29, 5684, 29
Lengths of input lists are 5684, 29, 29, 29, 30, 29, 5684, 30
Total time: 944.2424945831299s
Train Epoch: 30 [0/50000 (0%)]	Loss: 0.051110
param norm: 1895.007568359375

Test set: Avg. loss: 0.3672, Accuracy: 9232/10000 (92%)

Saving data...
 Lengths of SAVED lists are 5880, 30, 30, 30, 30, 30, 5880, 30
Lengths of input lists are 5880, 30, 30, 30, 31, 30, 5880, 31
Total time: 977.4595477581024s
Train Epoch: 31 [0/50000 (0%)]	Loss: 0.027973
param norm: 1895.4425048828125

Test set: Avg. loss: 0.3691, Accuracy: 9243/10000 (92%)

Saving data...
 Lengths of SAVED

In [ ]:
# ERROR DEBUGING
#import numpy as np
#A = np.load(error_write_path + '/m_aa_when_err.npy')

In [ ]:
#np.linalg.matrix_rank(A)